In [2]:
#Import Libraries
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, PULP_CBC_CMD

In [4]:
#Defining Optimization Problem as Minimization
model = LpProblem("Logistic_Optimization", LpMinimize)

In [6]:
#Data Input
plants = ["P1", "P2", "P3"]
warehouses = ["WH1", "WH2"]
customers = ["C1", "C2", "C3"]

#Plants Production Capacity and Costs
production_capacity = {"P1": 700, "P2": 600, "P3": 800}
production_cost = {"P1": 12, "P2": 10, "P3": 11}

# Transportation Costs (Plant to Warehouse)
plant_to_warehouse_cost = {
    ("P1", "WH1"): 7, ("P1", "WH2"): 5,
    ("P2", "WH1"): 5, ("P2", "WH2"): 6,
    ("P3", "WH1"): 6, ("P3", "WH2"): 8,
}

# Warehouse Storage Capacities and Holding Costs
storage_capacity = {"WH1": 1000, "WH2": 1200}
holding_cost = {"WH1": 2, "WH2": 3}

# Transportation Costs (Warehouse to Customer)
warehouse_to_customer_cost = {
    ("WH1", "C1"): 6, ("WH1", "C2"): 8, ("WH1", "C3"): 9,
    ("WH2", "C1"): 5, ("WH2", "C2"): 7, ("WH2", "C3"): 10,
}

# Customer Demand
demand = {"C1": 500, "C2": 700, "C3": 600}

In [8]:
# Decision Variables for Shipments from Plants to Warehouses
x_plant_to_warehouse = LpVariable.dicts(
    "Plant_to_Warehouse",
    [(p, w) for p in plants for w in warehouses],
    lowBound=0,
    cat="Integer"
)

# Decision Variables for Shipments from Warehouses to Customers
x_warehouse_to_customer = LpVariable.dicts(
    "Warehouse_to_Customer",
    [(w, c) for w in warehouses for c in customers],
    lowBound=0,
    cat="Integer"
)


In [10]:
# Objective Function
model += lpSum(
    production_cost[p] * lpSum(x_plant_to_warehouse[(p, w)] for w in warehouses) for p in plants
) + lpSum(
    plant_to_warehouse_cost[(p, w)] * x_plant_to_warehouse[(p, w)] for p in plants for w in warehouses
) + lpSum(
    warehouse_to_customer_cost[(w, c)] * x_warehouse_to_customer[(w, c)] for w in warehouses for c in customers
) + lpSum(
    holding_cost[w] * lpSum(x_warehouse_to_customer[(w, c)] for c in customers) for w in warehouses
)


In [12]:
#Production Capacity Constraints
for p in plants:
    model += lpSum(x_plant_to_warehouse[(p, w)] for w in warehouses) <= production_capacity[p]


In [16]:
#Storage Capacity Constraints
for w in warehouses:
    model += lpSum(x_plant_to_warehouse[(p, w)] for p in plants) >= lpSum(x_warehouse_to_customer[(w, c)] for c in customers)
    model += lpSum(x_plant_to_warehouse[(p, w)] for p in plants) <= storage_capacity[w]


In [18]:
#Demand Constraints
for c in customers:
    model += lpSum(x_warehouse_to_customer[(w, c)] for w in warehouses) == demand[c]


In [20]:
#Solving the Model
model.solve()

#Results
print("Status:", model.status)
print("Optimal Total Cost:", value(model.objective))

for var in model.variables():
    if var.value() > 0:  # Only display non-zero values
        print(f"{var.name}: {var.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/a17c54c257944fe6bb7b5257d1a64527-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/a17c54c257944fe6bb7b5257d1a64527-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 89 RHS
At line 101 BOUNDS
At line 114 ENDATA
Problem MODEL has 11 rows, 12 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 47100 - 0.00 seconds
Cgl0004I processed model has 7 rows, 8 columns (8 integer (0 of which binary)) and 22 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 47100 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search comple

In [22]:
#Transportation Cost Sensitivity
print("\nSensitivity Analysis: Transportation Costs")
scenarios = [("P1-WH1", 9), ("P2-WH2", 7)]  # Modify specific costs
for scenario, new_cost in scenarios:
    # Updating transportation cost
    plant_to_warehouse_cost[("P1", "WH1")] = new_cost
    model.solve()
    print(f"Scenario ({scenario} = {new_cost}): Total Cost = {value(model.objective)}")


Sensitivity Analysis: Transportation Costs
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/fcc9b5245855458a85be108384bbb246-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/fcc9b5245855458a85be108384bbb246-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 89 RHS
At line 101 BOUNDS
At line 114 ENDATA
Problem MODEL has 11 rows, 12 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 47100 - 0.00 seconds
Cgl0004I processed model has 7 rows, 8 columns (8 integer (0 of which binary)) and 22 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 47100 found by DiveCoefficient after 0 iterations and 0

In [24]:
#Customer Demand Sensitivity
print("\nSensitivity Analysis: Customer Demand")
demand_scenarios = [("C1", 500), ("C2", 600)]
for customer, new_demand in demand_scenarios:
    # Updating customer demand
    demand[customer] = new_demand
    model.solve()
    print(f"Scenario ({customer} Demand = {new_demand}): Total Cost = {value(model.objective)}")


Sensitivity Analysis: Customer Demand
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/96c6a73007c64c3886b63a365411ee1a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/96c6a73007c64c3886b63a365411ee1a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 89 RHS
At line 101 BOUNDS
At line 114 ENDATA
Problem MODEL has 11 rows, 12 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 47100 - 0.00 seconds
Cgl0004I processed model has 7 rows, 8 columns (8 integer (0 of which binary)) and 22 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 47100 found by DiveCoefficient after 0 iterations and 0 node

In [26]:
#Plant Capacity Sensitivity
print("\nSensitivity Analysis: Plant Capacity")
capacity_scenarios = [("P1", 500), ("P2", 700)]
for plant, new_capacity in capacity_scenarios:
    # Update plant capacity
    production_capacity[plant] = new_capacity
    model.solve()
    print(f"Scenario ({plant} Capacity = {new_capacity}): Total Cost = {value(model.objective)}")


Sensitivity Analysis: Plant Capacity
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/8ed5abf190744a3aa294430b39418d83-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/71/_d267zsd097gthqh_cpfxlt40000gn/T/8ed5abf190744a3aa294430b39418d83-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 89 RHS
At line 101 BOUNDS
At line 114 ENDATA
Problem MODEL has 11 rows, 12 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 47100 - 0.00 seconds
Cgl0004I processed model has 7 rows, 8 columns (8 integer (0 of which binary)) and 22 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 47100 found by DiveCoefficient after 0 iterations and 0 nodes